In [1]:

import matplotlib.pyplot as plt
import numpy as np
import time

# Import Sionna RT components
from sionna.rt import load_scene, Transmitter, Receiver, PlanarArray, CoverageMap

# For link-level simulations
from sionna.channel import cir_to_ofdm_channel, cir_to_time_channel,subcarrier_frequencies, OFDMChannel, ApplyOFDMChannel, CIRDataset
from sionna.nr import PUSCHConfig, PUSCHTransmitter, PUSCHReceiver
from sionna.utils import compute_ber, ebnodb2no, PlotBER
from sionna.ofdm import KBestDetector, LinearDetector
from sionna.mimo import StreamManagement

#matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time
import sionna
# Import Sionna RT components
from sionna.rt import load_scene, Transmitter, Receiver, PlanarArray, Antenna

# For link-level simulations
from sionna.channel import OFDMChannel

from sionna.nr import PUSCHConfig, PUSCHTransmitter, PUSCHReceiver
from sionna.utils import compute_ber, ebnodb2no, PlotBER
from sionna.ofdm import KBestDetector, LinearDetector
from sionna.mimo import StreamManagement

In [2]:
scene = load_scene(sionna.rt.scene.munich) # Try also sionna.rt.scene.etoile

In [3]:
scene.frequency = 3.5e9 # in Hz; implicitly updates RadioMaterials

scene.synthetic_array = True # If set to False, ray tracing will be done per antenna element (slower for large arrays)

In [5]:
scene.remove("tx")
scene.remove("rx")
scene.remove("tx1")
# Configure antenna array for all transmitters
scene.tx_array = PlanarArray(num_rows=8,
                             num_cols=8,
                             vertical_spacing=2.0,
                             horizontal_spacing=0.5,
                             pattern="tr38901",
                             polarization="V")

# Configure antenna array for all receivers
scene.rx_array = PlanarArray(num_rows=1,
                             num_cols=1,
                             vertical_spacing=2.0,
                             horizontal_spacing=0.5,
                             pattern="tr38901",
                             polarization="cross")

# Create transmitter


tx = Transmitter(name="tx",
                 position=[500,220,30])
# Create a receiver
scene.add(tx)

rx = Receiver(name="rx",
                 position=[50,50,30])

# Add transmitter instance to scene
scene.add(rx)


 # Transmitter points towards receiver

In [ ]:
paths = scene.compute_paths(max_depth=0) 
# 取得 Tx 到 Rx 的 CIR
subcarrier_spacing = 30e3
fft_size = 408
print("Shape of `a` before applying Doppler shifts: ", paths.a.shape)

# Apply Doppler shifts
paths.apply_doppler(sampling_frequency=subcarrier_spacing, # Set to 15e3 Hz
                    num_time_steps=200, # Number of OFDM symbols
                    tx_velocities=[3.,0,0], # We can set additional tx speeds
                    rx_velocities=[0,7.,0]) # Or rx speeds

print("Shape of `a` after applying Doppler shifts: ", paths.a.shape)

a, tau = paths.cir()
print("Shape of tau: ", tau.shape)

# Compute frequencies of subcarriers and center around carrier frequency
frequencies = subcarrier_frequencies(fft_size, subcarrier_spacing)

# Compute the frequency response of the channel at frequencies.
h_freq_no_jamming = cir_to_ofdm_channel(frequencies,
                             a,
                             tau,
                             normalize=True) # Non-normalized includes path-loss

# Verify that the channel power is normalized


print("Shape of h_freq_no_jamming: ", h_freq_no_jamming.shape)

In [ ]:
scene.preview(paths =paths)

In [6]:
cm = scene.coverage_map(max_depth=5,
                        diffraction=True,
                        cm_cell_size=(5., 5.),
                        num_samples=int(20e6)) 

In [8]:
scene.remove("tx1")

tx1 = Transmitter(name="tx1",
                 position=[8.5,21,27])
# Add transmitter instance to scene
scene.add(tx1)


In [9]:
scene.remove("tx")
cm2 = scene.coverage_map() 

In [ ]:
paths = scene.compute_paths(max_depth=0) 
# 取得 Tx 到 Rx 的 CIR
subcarrier_spacing = 30e3
fft_size = 408
print("Shape of `a` before applying Doppler shifts: ", paths.a.shape)

# Apply Doppler shifts
paths.apply_doppler(sampling_frequency=subcarrier_spacing, # Set to 15e3 Hz
                    num_time_steps=200, # Number of OFDM symbols
                    tx_velocities=[3.,0,0], # We can set additional tx speeds
                    rx_velocities=[0,7.,0]) # Or rx speeds

print("Shape of `a` after applying Doppler shifts: ", paths.a.shape)

a, tau = paths.cir()
print("Shape of tau: ", tau.shape)

# Compute frequencies of subcarriers and center around carrier frequency
frequencies = subcarrier_frequencies(fft_size, subcarrier_spacing)

# Compute the frequency response of the channel at frequencies.
h_freq_jamming = cir_to_ofdm_channel(frequencies,
                             a,
                             tau,
                             normalize=True) # Non-normalized includes path-loss

# Verify that the channel power is normalized


print("Shape of h_freq_jamming: ", h_freq_jamming.shape)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def angle_delay_transform(h_freq):
    """
    - 先對 subcarriers (頻域) 做 IFFT -> 時延域 (Delay)
    - 再對 rx_antenna (空域) 做 FFT -> 角度域 (AOA)
    """
    # Step 1: IFFT over subcarriers (Convert to Delay domain)
    h_delay = np.fft.ifft(h_freq, axis=-1)

    # Step 2: FFT over rx_antenna (Convert to Angle domain)
    h_angle_delay = np.fft.fftshift(np.fft.fft(h_delay, axis=0), axes=0)

    return h_angle_delay


# **1. 取得「無干擾」場景的 CSI**
h_tx1_no_jamming = h_freq_no_jamming.numpy()[0, 0, :, 0, 0, 0, :]  # Tx1 in No Jamming scenario

# **2. 取得「有干擾」場景的 CSI**
h_tx1_jamming = h_freq_jamming.numpy()[0, 0, :, 0, 0, 0, :]  # Tx1 in Jamming scenario
h_tx2_jamming = h_freq_jamming.numpy()[0, 0, :, 1, 0, 0, :]  # Tx2 (Jammer) in Jamming scenario

# **3. 計算合併通道 (有干擾場景的 Rx 端接收通道)**
h_combined = h_tx1_jamming + h_tx2_jamming  # Rx 端實際接收的總信號

# **4. 進行 Angle-Delay 轉換**
H_no_jam_ad = angle_delay_transform(h_tx1_no_jamming)  # 無干擾 Tx1
H_jam_ad = angle_delay_transform(h_combined)           # 有干擾 (Tx1 + Tx2)

# **5. 繪製熱圖**
plt.figure(figsize=(12,5))

# **無干擾場景**
plt.subplot(1,2,1)
plt.imshow(20*np.log10(np.abs(H_no_jam_ad)+1e-9), aspect='auto', cmap='jet')
plt.colorbar(label="Magnitude (dB)")
plt.xlabel("Delay Index")
plt.ylabel("Angle Index")
plt.title("AOA vs. Delay (No Jamming)")

# **有干擾場景**
plt.subplot(1,2,2)
plt.imshow(20*np.log10(np.abs(H_jam_ad)+1e-9), aspect='auto', cmap='jet')
plt.colorbar(label="Magnitude (dB)")
plt.xlabel("Delay Index")
plt.ylabel("Angle Index")
plt.title("AOA vs. Delay (Jamming)")

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # 引入 3D 繪圖工具

def angle_delay_transform(h_freq):
    """
    - 對子載波 (頻域) 做 IFFT 轉換到時延域
    - 對接收天線 (空域) 做 FFT 轉換到角度域 (AOA)
    """
    # Step 1: IFFT over subcarriers (Convert to Delay domain)
    h_delay = np.fft.ifft(h_freq, axis=-1)
    # Step 2: FFT over rx_antenna (Convert to Angle domain) 並 fftshift 使 0° 對齊中心
    h_angle_delay = np.fft.fftshift(np.fft.fft(h_delay, axis=0), axes=0)
    return h_angle_delay

# -------------------------------
# 假設你已有以下資料 (請自行替換為你的資料)
# -------------------------------
# 無干擾場景下 Tx1 的 CSI：shape (rx_antenna, subcarriers)
h_tx1_no_jamming = h_freq_no_jamming.numpy()[0, 0, :, 0, 0, 0, :]

# 有干擾場景下，分別取 Tx1 與 Tx2，並疊加得到 Rx 端合成通道
h_tx1_jamming = h_freq_jamming.numpy()[0, 0, :, 0, 0, 0, :]
h_tx2_jamming = h_freq_jamming.numpy()[0, 0, :, 1, 0, 0, :]
h_combined = h_tx1_jamming + h_tx2_jamming

# 進行 Angle-Delay 轉換
H_no_jam_ad = angle_delay_transform(h_tx1_no_jamming)  # 無干擾 Tx1
H_jam_ad    = angle_delay_transform(h_combined)         # 有干擾 (Tx1 + Tx2)

# 將幅度轉換為 dB 值
Z_no_jam = 20 * np.log10(np.abs(H_no_jam_ad) + 1e-9)
Z_jam    = 20 * np.log10(np.abs(H_jam_ad) + 1e-9)

# 建立網格，假設 X: Delay Index，Y: Angle Index
num_angles, num_delays = Z_no_jam.shape
X, Y = np.meshgrid(np.arange(num_delays), np.arange(num_angles))

# -------------------------------
# 3D 繪圖
# -------------------------------
fig = plt.figure(figsize=(14, 6))

# 3D 圖 - 無干擾場景
ax1 = fig.add_subplot(121, projection='3d')
surf1 = ax1.plot_surface(X, Y, Z_no_jam, cmap='jet', edgecolor='none')
ax1.set_title("AOA vs. Delay for Tx1 (No Jamming)")
ax1.set_xlabel("Delay Index")
ax1.set_ylabel("Angle Index")
ax1.set_zlabel("Magnitude (dB)")
fig.colorbar(surf1, ax=ax1, shrink=0.5, aspect=10)

# 3D 圖 - 有干擾場景
ax2 = fig.add_subplot(122, projection='3d')
surf2 = ax2.plot_surface(X, Y, Z_jam, cmap='jet', edgecolor='none')
ax2.set_title("AOA vs. Delay (Tx1 + Tx2 Combined)")
ax2.set_xlabel("Delay Index")
ax2.set_ylabel("Angle Index")
ax2.set_zlabel("Magnitude (dB)")
fig.colorbar(surf2, ax=ax2, shrink=0.5, aspect=10)

plt.tight_layout()
plt.show()


In [ ]:
scene.preview(paths = paths)

In [ ]:
energy_ratio = np.abs(H_jam_ad) / (np.abs(H_no_jam_ad) + 1e-9)
plt.imshow(20*np.log10(energy_ratio), aspect='auto', cmap='jet', vmin=-10, vmax=10)
plt.colorbar(label="Energy Change (dB)")
plt.title("Energy Ratio (Jamming vs. No Jamming)")
plt.show()


In [ ]:
#MUSIC spertum
import numpy as np
import matplotlib.pyplot as plt

def music_spectrum(H, num_sources, scan_angles, d=0.5, wavelength=1):
    """
    利用 MUSIC 演算法計算 MUSIC 頻譜
    H: 資料矩陣，形狀 (N, M)，N 為天線數，M 為 snapshot 數
    num_sources: 預設信號來源數 (例如主信號+干擾 = 2)
    scan_angles: 掃描角度範圍 (單位: 度)
    d: 天線間距（以波長為單位，預設 0.5）
    wavelength: 波長（預設 1）
    返回：
      spectrum: MUSIC 頻譜值，對應 scan_angles 中每個角度
    """
    # 計算接收協方差矩陣
    R = np.dot(H, H.conj().T) / H.shape[1]
    # 特徵值分解
    eigenvalues, eigenvectors = np.linalg.eig(R)
    # 按照特徵值大小排序（降序）
    idx = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]
    N = H.shape[0]
    # 假設信號子空間維度 = num_sources，噪聲子空間為剩下的維度
    noise_subspace = eigenvectors[:, num_sources:]
    
    # 掃描各個角度
    P = []
    for theta in scan_angles:
        n = np.arange(N)
        # 導向向量 a(theta)
        steering = np.exp(-1j * 2 * np.pi * d * n * np.sin(np.deg2rad(theta)) / wavelength)
        steering = steering.reshape(-1, 1)
        # MUSIC 頻譜計算公式：1 / |a^H E_n E_n^H a|
        spectrum_val = 1 / np.abs(np.conj(steering).T @ noise_subspace @ noise_subspace.conj().T @ steering)
        P.append(spectrum_val[0, 0])
    return np.array(P)

# -----------------------------------------
# 假設你已有以下 CSI 資料變數：
# h_freq_no_jamming: shape (1, 1, 64, 1, 64, 14, 408)
# h_freq_jamming: shape (1, 1, 64, 1, 64, 14, 408)
# -----------------------------------------

# 選擇一個固定的子載波索引（例如 200）作為 snapshot 的頻域點
subcarrier_index = 200

# 從無干擾場景中提取資料：
# 這裡我們取 [batch, rx_num, rx_antenna, tx_num, tx_antenna, OFDM_symbols, subcarrier]
# 選擇 tx_num=0 (Tx1)，並取全部 OFDM 符號 (維度 index 5) → 最終形狀 (64, 14)
H_no_jam = h_freq_no_jamming.numpy()[0, 0, :, 0, 0, :, subcarrier_index]  # (64, 14)

# 從有干擾場景中提取資料：
# 分別提取 tx_num=0 (主信號) 與 tx_num=1 (干擾源)
H_tx1_jam = h_freq_jamming.numpy()[0, 0, :, 0, 0, :, subcarrier_index]  # (64, 14)
H_tx2_jam = h_freq_jamming.numpy()[0, 0, :, 1, 0, :, subcarrier_index]  # (64, 14)
# 將兩者線性疊加，得到 Rx 端合成通道
H_jam = H_tx1_jam + H_tx2_jam  # (64, 14)

# 設定 MUSIC 參數：
num_sources = 2  # 假設有 2 個入射路徑 (主信號與干擾)
scan_angles = np.linspace(-90, 90, 181)  # 從 -90° 到 90°

# 分別計算無干擾與有干擾的 MUSIC 頻譜
spectrum_no_jam = music_spectrum(H_no_jam, num_sources, scan_angles)
spectrum_jam = music_spectrum(H_jam, num_sources, scan_angles)

# 繪製 MUSIC 頻譜（以 dB 標度顯示，並正規化）
plt.figure(figsize=(12,6))

plt.subplot(2,1,1)
plt.plot(scan_angles, 20*np.log10(np.abs(spectrum_no_jam) / np.max(np.abs(spectrum_no_jam))), label="No Jamming")
plt.xlabel("Angle (degrees)")
plt.ylabel("Spectrum (dB)")
plt.title("MUSIC Spectrum - No Jamming")
plt.grid(True)
plt.legend()

plt.subplot(2,1,2)
plt.plot(scan_angles, 20*np.log10(np.abs(spectrum_jam) / np.max(np.abs(spectrum_jam))), label="Jamming", color='r')
plt.xlabel("Angle (degrees)")
plt.ylabel("Spectrum (dB)")
plt.title("MUSIC Spectrum - Jamming")
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
cm_tensor = 
cm2_tensor = cm2.rss
print("Coverage Map Tensor Shape:", cm_tensor.shape)
print("Coverage Map Tensor Shape:", cm2_tensor.shape)

AttributeError: 'CoverageMap' object has no attribute 'path_gain'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 轉換 Tensor 為 NumPy 陣列
cm_numpy = cm_tensor.numpy()  # Tx1 的 CoverageMap
cm2_numpy = cm2_tensor.numpy()  # Tx2 的 CoverageMap

# 計算 Tx1 和 Tx2 的覆蓋範圍
cm_combined = np.mean(cm_numpy, axis=0)  # Tx1 覆蓋
cm2_combined = np.mean(cm2_numpy, axis=0)  # Tx2 覆蓋
cm_diff = cm_combined - cm2_combined  # 訊號強度的差異

# 統一 Colorbar 範圍
vmin = min(cm_combined.min(), cm2_combined.min(), cm_diff.min())
vmax = max(cm_combined.max(), cm2_combined.max(), cm_diff.max())
threshold = 1e-9  # 你可自行調整
masked_cm1  = np.ma.masked_less(cm_combined,  threshold)
masked_cm2  = np.ma.masked_less(cm2_combined, threshold)
# 創建 1x3 子圖
fig, axes = plt.subplots(1, 3, figsize=(18, 6), constrained_layout=True)

# 第一張：Tx1 覆蓋範圍
im1 = axes[0].imshow(masked_cm1, cmap="jet", origin="lower", vmin=-abs(vmax/5), vmax=abs(vmax/5))
im1.cmap.set_bad(color='white')  # 遮罩的值(背景小於 threshold) 就用白色
axes[0].set_title("Coverage Map (Tx1)")
axes[0].set_xlabel("X-axis (Grid)")
axes[0].set_ylabel("Y-axis (Grid)")

# 第二張：Tx2 覆蓋範圍
im2 = axes[1].imshow(masked_cm2, cmap="jet", origin="lower", vmin=-abs(vmax/5), vmax=abs(vmax/5))
axes[1].set_title("Coverage Map (Tx2)")
axes[1].set_xlabel("X-axis (Grid)")
axes[1].set_ylabel("Y-axis (Grid)")


# 第三張：Tx1 - Tx2 差異
im3 = axes[2].imshow(cm_diff, cmap="bwr", origin="lower", vmin=-abs(vmax/5), vmax=abs(vmax/5))
axes[2].set_title("Coverage Difference (Tx1 - Tx2)")
axes[2].set_xlabel("X-axis (Grid)")
axes[2].set_ylabel("Y-axis (Grid)")

# 設定 colorbar 放在圖的右側，不擋住圖片
cbar = fig.colorbar(im3, ax=axes.ravel().tolist(), location="right", shrink=0.7, pad=0.02)
cbar.set_label("Coverage Value")

plt.show()
print(vmax)